using embeddings to convert words into a T x D matrix of numbers<br>
** read about word3vec and GloVe

<ol>
<li>Convert words into integers (indices into the word embedding matrix)</li>
<li>Mapping from words to integer</li>
<li>integers start from 1 not 0 because 0 is used for padding</li>
<li>convert strings to sequences</li>
<li>Tokenize: each string is a single word</li>
<li>The process is string to tokens to integers to vectors</li>
<li>Padding: to make sequences the same length</li>
<li>N is the number of lists of integers</li>
<li>T is the length of the sequence</li>
<li>Truncating pre or post</li>
<li>Padding: pre or post</li>
</ol>

We have an N x T matrix of word indices. When passed by the embedding layer, we get N x T x D tensor converting it into a word vector. 

In [20]:
# preprocessing text
import csv
from nltk.corpus import stopwords
import numpy as np 
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [5]:
# using list of stop words to remove from text
stop_words = stopwords.words("english")
print(len(stop_words))

153


In [6]:
# preprocessing text
texts = []
labels = []
# reading the data. 
# for each row, the first item is the label
with open('data/bbc-text.csv','r') as file: # reading the files
    data = csv.reader(file)
    next(data, None)
    for row in data:
        labels.append(row[0])
        sentence = row[1]
        for word in stop_words: # replacing tokens with an empty space, and then replacing double space with a single space
            token = ' ' + word + ' '
            sentence = sentence.replace(token,' ')
            sentence = sentence.replace('  ', ' ')
        texts.append(sentence)

        
print(len(labels))
print(len(texts))


2225
2225


In [19]:
MAX_VOCAB_SIZE = 10000
# out of words token
oov_tok = '<OOV>'
# padding
padding_type = 'post'
trunc_type = 'post'
max_len = 120

# embedding
embedding_dim = 16
tokenizer = Tokenizer(num_words = MAX_VOCAB_SIZE)
tokenizer.fit_on_texts(texts)
# a dictionary of words as key and integer as value
word_index = tokenizer.word_index
# convert to sequences
sequences = tokenizer.texts_to_sequences(texts)
# padding
padded = pad_sequences(sequences, maxlen=max_len ,truncating=trunc_type, padding=trunc_type)
print(f"Shape of padded sequences: {padded.shape}")

Shape of padded sequences: (2225, 120)


In [23]:
# tokenizing labels
label_tokenizer = Tokenizer()
label_tokenizer.fit_on_texts(labels)
label_index = label_tokenizer.word_index
print(label_index)
label_seq = np.array(label_tokenizer.texts_to_sequences(labels))
print(label_seq.shape)

{'sport': 1, 'business': 2, 'politics': 3, 'tech': 4, 'entertainment': 5}
(2225, 1)


In [34]:
# splitting the data
training_size = int(np.ceil(0.8 * len(texts)))
X_train, X_valid = padded[:training_size], padded[training_size:]
y_train, y_valid = label_seq[:training_size], label_seq[training_size:]
print(f"Shape of X_train {X_train.shape}")
print(f"Shape of X_valid {X_valid.shape}")
print(f"Shape of y_train {y_train.shape}")
print(f"Shape of y_valid {y_valid.shape}")

Shape of X_train (1780, 120)
Shape of X_valid (445, 120)
Shape of y_train (1780, 1)
Shape of y_valid (445, 1)


# Modeling

In [38]:
from tensorflow.keras.layers import Input, GlobalAveragePooling1D, Dense
from tensorflow.keras.layers import Embedding, LSTM
from tensorflow.keras.models import Model

In [ ]:
# model
i = Model(shape=(max_len)) # input layer